# Titanic: Survival Model

Build and train a model to predict survival on the Titanic using a [cleaned and split dataset](https://huggingface.co/datasets/jamieoliver/titanic-2410), and upload the model to Hugging Face.

Based on https://github.com/fastai/course22/blob/master/clean/05-linear-model-and-neural-net-from-scratch.ipynb using the dataset from https://www.kaggle.com/competitions/titanic.

Plan
- [x] Download [cleaned and split dataset](https://huggingface.co/datasets/jamieoliver/titanic-2410) from Hugging Face
- [x] Prepare data for model
    - [x] Load training dataset as PyTorch tensors
    - [x] Normalise training dataset
- [ ] Train model
- [ ] Test model
- [ ] Upload model to Hugging Face

##  Download Dataset from Hugging Face

In [1]:
from datasets import *

datasetDict = load_dataset('jamieoliver/titanic-2410')
datasetDict

DatasetDict({
    train: Dataset({
        features: ['PassengerId', 'Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'LogFare', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
        num_rows: 712
    })
    validation: Dataset({
        features: ['PassengerId', 'Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'LogFare', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
        num_rows: 179
    })
    test: Dataset({
        features: ['PassengerId', 'Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'LogFare', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
        num_rows: 418
    })
})

## Prepare Data for Model

### Load Training Dataset as PyTorch Tensors

In [2]:
from torch import *
from torch.utils.data import *

set_printoptions(linewidth=120, edgeitems=10)

In [3]:
train_dataset = datasetDict['train']
train_dataset

Dataset({
    features: ['PassengerId', 'Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'LogFare', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
    num_rows: 712
})

The dependent variable is the variable we are predicting i.e. `survived`.

In [4]:
dependent_var = tensor(train_dataset.to_pandas()['Survived'].values, dtype=float)
dependent_var

tensor([0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1.,
        0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0.,
        1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0.,
        1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
        0., 1., 1., 0., 1., 0., 0., 0., 

In [5]:
dependent_var.shape

torch.Size([712])

The independent variables are the variables we will use to make the prediction. Note that we use a trick in mutiplying the Pandas DataFrame by 1 to convert booleans to integers.

In [6]:
independent_cols = ['Age', 'SibSp', 'Parch', 'LogFare', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
                    'Embarked_C', 'Embarked_Q', 'Embarked_S']

independent_vars = tensor((train_dataset.to_pandas()*1)[independent_cols].values, dtype=float)
independent_vars

tensor([[22.0000,  0.0000,  0.0000,  2.0949,  0.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000],
        [24.0000,  2.0000,  0.0000,  4.3108,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000],
        [24.0000,  0.0000,  0.0000,  5.4316,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000,  1.0000,  0.0000,  0.0000],
        [32.0000,  0.0000,  0.0000,  4.3476,  1.0000,  0.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000],
        [30.0000,  0.0000,  0.0000,  2.1102,  0.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000],
        [18.0000,  0.0000,  2.0000,  2.6391,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000],
        [47.0000,  1.0000,  1.0000,  3.9807,  1.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000],
        [34.0000,  0.0000,  0.0000,  2.4423,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000],
        [39.0000,  1.0000,  1.0000,  4.7175,  1.0000,  0

In [7]:
independent_vars.shape

torch.Size([712, 12])

### Normalise Training Dataset

In [8]:
max_vals,indices = independent_vars.max(dim=0)
independent_vars /= max_vals
independent_vars

tensor([[0.2973, 0.0000, 0.0000, 0.3357, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000],
        [0.3243, 0.2500, 0.0000, 0.6907, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000],
        [0.3243, 0.0000, 0.0000, 0.8703, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000],
        [0.4324, 0.0000, 0.0000, 0.6966, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000],
        [0.4054, 0.0000, 0.0000, 0.3381, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000],
        [0.2432, 0.0000, 0.3333, 0.4229, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000],
        [0.6351, 0.1250, 0.1667, 0.6378, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000],
        [0.4595, 0.0000, 0.0000, 0.3913, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000],
        [0.5270, 0.1250, 0.1667, 0.7559, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000],
        [0.6757, 0.0000, 0.1667, 0.8838, 1.000